In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
pd.read_pickle("failed_experiments.pkl").columns

In [ ]:
# df = pd.read_pickle("results.pkl")
df = pd.concat([pd.read_pickle("Multiple Pairs Results.pkl"), pd.read_pickle("Single Pairs Results.pkl")])
df["nodes"] = df["nodes"].astype(str)
df["experiment_type"] = df["experiment_type"].map(lambda x: "Single Pairs" if x == "Single" else x)
df["perturber_class"] = df["perturber_class"].map(lambda x: "GreedyMin" if x == "MinFirst" else x)
df.columns

In [ ]:
print(len(df), sum(df["Success"]), sum(df["Success"])/len(df))
px.histogram(df, x="perturber_class", color="Status",facet_col="experiment_type",hover_data=["LP Status"], category_orders={"perturber_class": ["PathAttack", "GreedyFirst", "GreedyMin"]})

In [ ]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x) if len(x)>0 else None)
df["Total Add Time"] = df["Add Times"].map(sum)
df["Total Perturb Time"] = df["Perturb Times"].map(sum)

In [ ]:
df.to_csv("results.csv")

In [ ]:
index_cols = ["nodes", "condition_index", "configuration_index"]
global_cols = ["epsilon", "n_nodes_per_experiment", "n_experiments", "n_trials", "min_path_length", "use_multithreading", "global_budget", "local_budget"]
extra_cols = ["Status","Add Times", "Perturb Times", "Perturbation Dict","source", "target", "LP Status", "IIS_paths", "IIS_edges", "IIS_global_budget", "Supporting Paths", "Global Budget Slack"]
config_cols = ["k", "perturber_class", "graph_name", "weights", "experiment_type"]
data_cols = ["Time Taken", "Iterations", "Number of Paths", "Number of Edges", "Original Distance", "Final Distance", "Success", "Total Perturbations","Total Add Time", "Total Perturb Time"]

In [ ]:
# Average Over Trials
a = df[index_cols+data_cols].groupby(index_cols).mean()
a = a.join(df.set_index(index_cols)[config_cols]) # Add back config information
a = a.drop_duplicates()

In [ ]:
## TODO: Instances where the LP is infeasible or runs out of iterations should be removed from the data

for y_val in ["Time Taken", "Total Perturbations", "Number of Paths"]:
    for experiment_type in a["experiment_type"].unique():
        if a.empty: continue
        fig = px.box(a[a["experiment_type"] == experiment_type], 
                    title=f"{experiment_type}: {y_val}",
                    # x="nodes", 
                    y=y_val, 
                    boxmode="group", 
                    color="perturber_class", 
                    facet_col="weights", 
                    facet_row="k",  
                    height=1000,
                    hover_data=data_cols,
                    category_orders={"Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                    "Success": [True, False]},
                    points="all",
                    # boxpoints=False,
                    )
        fig.update_xaxes(visible=False, matches=None)
        # fig.update_yaxes(range=[-1,13])
        # fig.update_traces(boxmean=True) 
        # fig.update_xaxes(visible=True)
        fig.show()

In [ ]:
b = a.reset_index().set_index(["configuration_index", "condition_index"])

In [ ]:

for experiment_type in a.experiment_type.unique():
    fig = px.bar(b[b["experiment_type"]==experiment_type], 
                title=f"{experiment_type} Results",
                x="nodes", 
                y="Time Taken", 
                barmode="group", 
                color="perturber_class", 
                facet_col="weights", 
                facet_row="k", 
                pattern_shape="Success", 
                # height=1000,
                # hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations", 'Final Distance',"Status", "LP Status", "nodes"],
                category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
                                "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                "Success": [True, False]}
                )
    fig.update_xaxes(visible=False)
    fig.update_xaxes(matches=None)
    # fig.update_xaxes(visible=True)
    fig.show()